In [34]:
import torch
from torchtext import data
from torchtext.legacy import data

from pathlib import Path

import pandas as pd
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from google_drive_downloader import GoogleDriveDownloader as gdd
from torch.utils.data import DataLoader, Dataset
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm, tqdm_notebook

import random
import matplotlib.pyplot as plt
import seaborn as sns
import re
#from torchtext.legacy.data import Field
#from torchtext.legacy.data.Field

ModuleNotFoundError: No module named 'torchtext.legacy'

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


device(type='cpu')

In [3]:
DATA_PATH = 'C:/Users/10694387/Downloads/archive/IMDB Dataset.csv'
df = pd.read_csv(DATA_PATH)

In [4]:
df.sample(5)

,review,sentiment
21231,"I loved ""Dan in Real Life"". A wonderful journe...",positive
44363,This is a great movie. When two people from di...,positive
40592,I'm rarely moved to make a comment online abou...,negative
16240,Like Steven Seagal I also am a big Van Damme f...,positive
11473,This derivative erotic thriller remains watcha...,negative


In [7]:
def labels_distribution(y):
    fig = plt.figure(figsize=(8,5))
    ax = sns.barplot(x=y.unique(),y=y.value_counts());
    ax.set(xlabel='Labels')
    print(y.value_counts())

In [18]:
def tokenizer(texts):
    char_based = True
    if char_based:
        tokenizer = lambda s: list(s) # char-based
    else:
        tokenizer = lambda s: s.split() # word-based
    return tokenizer

In [8]:
def cleanup_text(texts):
    cleaned_text = []
    for text in texts:
        # remove punctuation
        text = re.sub('[!#?,.:";]', ' ', text)
        # remove multiple spaces
        text = re.sub(r' +', ' ', text)
        # remove newline
        text = re.sub(r'\n', ' ', text)
        cleaned_text.append(text)
    return cleaned_text

In [9]:
def get_files(path, dev_size, max_document_length, seed, data_type, tokenizer):
    # include_lengths = True - This will cause batch.text to now be a tuple with the first element being our sentence (a numericalized tensor that has been padded) and the second element being the actual lengths of our sentences.
    Text = data.Field(preprocessing=cleanup_text, tokenize=tokenizer, batch_first=True, include_lengths=True, fix_length=max_document_length) # fix_length - make the sentences padded in the same lengths for all the batches
    Label = data.Field(sequential=False, use_vocab=False, pad_token=None, unk_token=None)

    # All files:
    fields = [('text', Text), ('labels', Label)]

    train_data, test_data = data.TabularDataset.splits(
        path=path,
        train= data_type + '_train.tsv',
        test= data_type + '_test.tsv',
        format='tsv',
        fields=fields,
        skip_header=False
    )

    train_data, valid_data = train_data.split(split_ratio=dev_size, random_state=random.seed(seed))
    print(f'Number of training examples: {len(train_data)}')
    print(f'Number of validation examples: {len(valid_data)}')
    print(f'Number of testing examples: {len(test_data)}')
    return train_data, valid_data, test_data, Text, Label

In [31]:
train_data, valid_data, test_data, Text, Label = get_files(DATA_PATH, 20, 5000, 40, float, tokenizer)

train_data[0]
valid_data[0]
test_data[0]
Text[0]
Label[0]

AttributeError: module 'torchtext.data' has no attribute 'Field'